___
# Ciência dos Dados - PROJETO 2

___
## Guilherme Leite

## Leonardo Neves

___

___

## 1. Problema

O Classificador Naive-Bayes, o qual se baseia no uso do teorema de Bayes, é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser SPAM considerando as palavras em seu conteúdo e, de forma complementar, permite calcular a probabilidade de uma mensagem ser HAM dada as palavras descritas na mensagem.

Para realizar o MVP (minimum viable product) do projeto, você precisa programar uma versão do classificador que "aprende" o que é uma mensagem SPAM considerando uma base de treinamento e comparar o desempenho dos resultados com uma base de testes. 


___
## 2. Separação da base de dados em Treinamento e Teste

A base de dados deve ser separada em duas partes, aleatoriamente, considerando: 
    
    75% dos dados para a parte Treinamento; e
    25% dos dados para a parte Teste.

In [39]:
import pandas as pd
import os
import numpy as np

In [40]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
C:\Users\Leonardo\OneDrive\insper\dps_2018\ciencia_dos_dados\projeto_2\DPCD-master\P2


In [73]:
df = pd.read_excel('spamham.xlsx') #le o arquivo

df = df.replace(',','',regex=True)
df = df.replace('@','',regex=True)
df = df.replace(';','',regex=True)
df = df.replace('  ','',regex=True)
df = df.replace(':','',regex=True)
df = df.replace('"','',regex=True)
df = df.replace('\\*','',regex=True)
df = df.replace('\*','',regex=True)
df = df.replace('_','',regex=True)
df = df.replace('\$','',regex=True)
df = df.replace('rt','',regex=True)
df = df.replace('//','',regex=True)
df = df.replace('/','',regex=True)
df = df.replace('#','',regex=True)
df = df.replace('\(','',regex=True)
df = df.replace('\)','',regex=True)
df = df.replace('\'','',regex=True)
df = df.replace('\?','',regex=True)
df = df.replace('\-','',regex=True)
df = df.replace('!','',regex=True)

df = df.sample(frac=1) #Deixa o df em uma ordem aleatoria

L = len(df)
Learn = int(L*0.75) #tamanho do df de treinamento
Test = int(L*0.25) # tamenho do df de teste

dfLearn = df.iloc[0:Learn] #df de treinamento
dfLearn = dfLearn.reset_index(drop=True)

dfTest = df.iloc[Learn:L] #df de teste
dfTest = dfTest.reset_index(drop=True)


In [74]:
Ham = []
Spam =[]
NSpam = 0

for index, row in dfLearn.iterrows(): #Percorre o df linha por linha
    if row['Class'] == 'spam':
        linha = row['Email']
        Linha = linha.split() #Separa as palavras do conteúdo do email
        NSpam +=1 #Adiciona +1 ao numero de emails de Spam
        for palavra in Linha:
            Hm = []
            for letra in palavra:  #Separa as letras das palavras
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra) 
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                Spam.append(hm)
    else:
        linha = str(row['Email'])
        Linha = linha.split()  #Separa as palavras do conteúdo do email
        for palavra in Linha:
            Hm = []
            for letra in palavra:
                if letra.isalpha() == True: # Se a letrea é alfabética:
                    Hm.append(letra)
            hm  = ''.join(Hm) #Junta as letras em palavra
            if len(hm)>1: # Filtra palavras com menos de duas Letras
                Ham.append(hm)

In [75]:
print(Spam[0:10])

['Hi', 'its', 'LUCY', 'Hubby', 'at', 'meetins', 'all', 'day', 'Fri', 'will']


___
## 3. Classificador Naive-Bayes

In [76]:
P_Spam = NSpam/len(dfLearn) #Probabilidade de uma mensagem ser SPAM 
P_Ham = 1-P_Spam #Probabilidade de uma mensagem ser HAM
print('Probabilidade de uma mensagem ser SPAM: ',P_Spam, 
      '\nProbabilidade de uma mensagem ser HAM: ',P_Ham)

Probabilidade de uma mensagem ser SPAM:  0.1255980861244019 
Probabilidade de uma mensagem ser HAM:  0.8744019138755981


In [77]:
d_ham =dfTest[dfTest.Class == 'ham']
d_spam =dfTest[dfTest.Class == 'spam']
d_ham.Email

0         No de.am seeing in online shop so that i asked.
1       Was actually sleeping and still might when u c...
5                                       Wat r u doing now
6       Am on a train back from nohampton so im afraid...
7       I REALLY NEED 2 KISS U I MISS U MY BABY FROM U...
8       Great. Have a safe trip. Dont panic surrender ...
9       Ya they are well and fine. BBDpooja full pimpl...
10                                 Watching ajith film ah
11      \Getting tickets 4 walsall tue 6 th march. My ...
12      Honeybee Said Im d Sweetest in d World God Lau...
13               Armand says get your ass over to epsilon
14      Hey im bored... So im thinking of u... So wat ...
15                       K..i deleted my contact that why
16      Hmmm ... I thought we said 2 hours slave not 3...
17      Babe Im answering you cant you see me  Maybe y...
18      Dunno cos i was v late n when i reach they ins...
19                        Customer place i will call you.
20      Indian

In [92]:
lista_ham = np.sum(d_ham.Email + " ").split()
vh= pd.Series(lista_ham)
vh.value_counts()
lista_spam = np.sum(d_spam.Email + " ").split()
vs= pd.Series(lista_spam)
vs.value_counts()
lista_spam.count('to')

184

In [102]:
lsp= len(lista_spam)
lha= len(lista_ham)
x = "SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info"
def email(x):
    spams = []
    hams = []
    sep=x.split(' ')
    for i in range(len(sep)):
        spams.append((lista_spam.count(sep[i])/lsp))
        hams.append((lista_ham.count(sep[i])/lha))
    pps = np.prod(spams)
    pph = np.prod(hams)
    if pph > pps:
        return "S"
    else:
        return "N"
email(x)

'N'

___
## 4. Curva ROC

___
## 5. Qualidade do Classificador alterando a base de treinamento